In [1]:
import numpy as np
import tensorflow as tf
import random
import plotly.graph_objects as go
import requests
import os
import umap

In [2]:
def read_file(file):
    with open(file, "r") as f:
        data = f.read()
    return data

In [3]:
API_KEY = read_file("keys/api_key.txt")
HMAC_KEY = read_file("keys/hmac_key.txt")

In [4]:
X_bytes = (requests.get('https://studio.edgeimpulse.com/v1/api/17190/dsp-data/22/x/training', headers={'x-api-key': API_KEY})).content
Y_bytes = (requests.get('https://studio.edgeimpulse.com/v1/api/17190/dsp-data/22/y/training', headers={'x-api-key': API_KEY})).content

In [5]:
if not os.path.exists("temp"):
    os.makedirs("temp")

with open('temp/x_train.npy', 'wb') as file:
    file.write(X_bytes)
with open('temp/y_train.npy', 'wb') as file:
    file.write(Y_bytes)
    
X = np.load('temp/x_train.npy')
Y = np.load('temp/y_train.npy')[:,0]

In [6]:
#X = X.reshape((-1,200,3))

In [7]:
X.shape

(1266, 600)

In [57]:
reducer = umap.UMAP(n_components=3, n_neighbors=15)

In [58]:
embedding = reducer.fit_transform(X)

In [59]:
embedding.shape

(1266, 3)

In [60]:
class_names = np.array(["A", "B", "C", "D", "E", "F"])

In [77]:
class_names

array(['A', 'B', 'C', 'D', 'E', 'F'], dtype='<U1')

In [50]:
class_names[Y-1]

array(['F', 'F', 'F', ..., 'B', 'B', 'B'], dtype='<U1')

In [85]:
data = go.Scatter3d(x=embedding[:,0], 
                    y=embedding[:,1], 
                    z=embedding[:,2],
                    text = class_names[Y-1],
                    mode="markers",
                    customdata = class_names[Y-1].reshape((-1,1)),
                    hovertemplate = '%{customdata[0]}',
                    marker = dict(
                       size=5,
                       color=Y,
                       showscale=True,
                       line = dict(
                            width=0.01,
                            color="LightGray"
                     )
                  )
            )
fig = go.Figure(data, layout=dict(
                                height=800,
                                title="UMAP 3D Representation of the training data"
                                )
                )
fig.show()